# WIN bot V0

# Author: Henrique Bittencourt Netto Monteiro

### 05/08/2020

In [ ]:
import os
import time
import pytz
import email
import imaplib
import numpy as np
import pandas as pd
import numexpr as ne
import MetaTrader5 as mt5
from datetime import datetime, date
from indicators import get_indicators
from smtplib import SMTP_SSL, SMTP_SSL_PORT

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.figsize'] = 15, 8

# Hyperparameters and global variables

In [ ]:
# RISK
limit = 3
exposure = 3
threshold = 500

deviation = 10

# SHUTDOWN TOLERANCE
tolerance_minutes = 5

In [ ]:
# TIME INFO
timezone = pytz.timezone("Etc/UTC")
utc_from = datetime(2025, 1, 1, tzinfo=timezone)

# EMAIL
email_address, email_password = 'email@provider.com', 'password'

# INDICATOR SPECS
maximum_length = 205

# TRADE DICT
trades = {"L": np.array([]), "S": np.array([])}

# STRAT DF
strat_dict = {}
df_strat = pd.read_csv("sample.csv")
for i in df_strat.index.to_numpy():
    strat_dict[i] = 0

# Initialization functions

In [ ]:
def log(message):
    datetime_now = datetime.now()
    timestamp = datetime_now.strftime('%Y-%m-%d %H:%M:%S')
    full_message = f"\n({timestamp}) {message}"
    print(full_message)
    
    day_now = datetime_now.strftime('%Y-%m-%d')
    filename = f"{day_now} WIN.txt"
    path = os.path.join("logs", filename)
    if filename not in os.listdir("logs"):
        f = open(path, "w+")
        f.write(f"({timestamp}) File created")
        f.close()
    f = open(path, "a+")
    f.write(full_message)
    f.close()    

def connect():
    if not mt5.initialize():
        log(f"initialize() failed, \nerror code = {mt5.last_error()}")
        quit()
    log(f"New connection established with MT5: \n {mt5.terminal_info()}")

In [ ]:
def get_contract(prefix):
    win_symbols = mt5.symbols_get(prefix)
    min_time, now = 1e12, time.time()
    for symbol in win_symbols:
        name = symbol.name
        info = mt5.symbol_info(name)
        if info != None:
            spread, digits, expiration = info.spread, info.digits, info.expiration_time
            remaining_days = int(expiration - time.time())//86400
            # if contract expires today, just trade the next
            if expiration > now and expiration < min_time and remaining_days > 0:
                min_time, next_win = expiration, name

    remaining_days = int(min_time - time.time())//86400
    if remaining_days > 68:
        raise Exception(f"Remaining days are high: {remaining_days}")
    log(f"Next contract is {next_win} with {remaining_days} remaining days")

    if remaining_days > 0:
        return next_win
    else:
        return None

# Functions for data acquisition

In [ ]:
def initialize_df(name):
    rates = mt5.copy_rates_from(name, mt5.TIMEFRAME_M1, utc_from, maximum_length+1)
    if rates is not None:
        df2, df3 = get_df(rates[:-1])
        
        if len(df2):
            if (sum(abs(df2["epoch"] - np.sort(df2["epoch"]))) != 0):
                raise Exception(f"Incosistency in epochs in df: {df2['epoch']}")
            current_epoch, current_time = max(df2["epoch"]), max(df2["time"])
        else:
            current_epoch, current_time = int(rates[-2][0]), "NONE"
    
    else:
        raise Exception(f"Could not get any data for {name}")
    
    log(f"Initialized df with {len(df2)} rows and last time {current_time}")
    return df2, df3, current_epoch

def get_last_tick(name):
    lasttick = mt5.symbol_info_tick(name)
    if lasttick is None:
        log(f"symbol_info_tick({name}) failed, \nerror code = ", mt5.last_error())
        raise Exception(f"ERROR: get_last_tick({name}) failed")
    return lasttick.ask, lasttick.bid

# Functions for data processing

In [ ]:
strat_names = ["SMA", "EMA", "SEN", "OBV", "STD", "ATR", "RSI_", "STOC", "AROON", "CCI", "ADX", "CMF", "MFI", "AO", "UO",
               "MACD", "SAR", "MEAN_CROSSOVER", "DEV_CROSSOVER", "MARTELO", "ROSS", "3BP"]
doubles = ["RSI_", "STOC", "CCI", "MFI", "CMF", "UO"]
deviation_range = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 23, 26, 30, 32, 35, 37, 40]

def get_numbers(array, name):
    for i, elem in enumerate(array):
        item = elem.replace("n", "-")
                
        if name == "ROSS" and i == 2:
            if float(item) > 4:
                item = float(item)/10
        elif len(item) > 1:
            if item[0] == "0":
                item = f"0.{item[1:]}"
            elif len(item) > 2 and item[0:2] == "-0":
                item = f"-0.{item[2:]}"
            
        item = int(item) if float(item) % 1 == 0 else round(float(item), 3)
        
        array[i] = item
    return array

def get_indicators_list(df_strat):
    strats_to_add = {"STD": deviation_range, "ATR": deviation_range, "ADX": deviation_range, "STOC": [[deviation_range, []]]}
    all_strats = df_strat.STRAT.to_numpy()
    for strat in all_strats:
        pieces = strat.split(" ")
        for piece in pieces:
            for name in strat_names:
                if name in piece:
                    if name not in strats_to_add:
                        strats_to_add[name] = []
                    spec = piece.replace(name, "").replace("X_", "").replace("BIN_", "")
                    parts = spec.split("_")
                    parts = get_numbers(parts, name)
                    if name in doubles:
                        if len(parts) == 2:
                            item = [[parts[0]], [parts[1]]]
                        else:
                            item = [[parts[0]], []]
                    else:
                        item = parts
                    if item not in strats_to_add[name]:
                        strats_to_add[name].append(item)

    means, devs = np.array([]), np.array([])
    for tipo in ["SMA", "EMA", "SEN"]:
        if tipo in strats_to_add:
            means = np.append(means, np.array(strats_to_add[tipo]).flatten())

    means = pd.Series(means).unique()
    if len(means):
        strats_to_add["SMA"] = list(means.astype(int))
        strats_to_add["EMA"] = list(means.astype(int))
        strats_to_add["SEN"] = list(means.astype(int))
        strats_to_add["MEAN_CROSSOVER"] = [list(means.astype(int))]

    strats_to_add["RSI"] = strats_to_add.pop("RSI_")

    indicators2 = []
    for key in list(strats_to_add.keys()):
        indicators2.append({"name": key, "params": strats_to_add[key]})
        
    return indicators2

indicators = get_indicators_list(df_strat)

In [ ]:
def update_indicators(df2):
    df3 = get_indicators(df2, indicators)
    return df3

In [ ]:
def get_df(rates, indicators=True):
    df2 = pd.DataFrame(rates)
    df2 = df2.rename(columns={"real_volume": "voltot", "time": "epoch"})
    
    df2["epoch"] = df2["epoch"].astype(int)
    df2["time"] = pd.to_datetime(df2["epoch"], unit='s')
    df2["date"] = pd.to_datetime(df2["time"]).apply(lambda x: date(x.year, x.month, x.day)).astype(str)
    df2["minute"] = pd.to_datetime(df2["time"]).apply(lambda x: 900 + 100*(x.hour-9) + x.minute).astype(int)
    df2["time"] = df2["time"].astype(str)
    df2 = df2[["date", "minute", "epoch", "time", "open", "low", "high", "close", "voltot"]]
    
    datetime_now = datetime.fromtimestamp(time.time())
    date_now = datetime_now.strftime("%Y-%m-%d")
    minute_now = int(datetime_now.strftime("%H%M"))
    df2 = df2[(df2.date == date_now) & (df2.minute >= 900)]
    
    # Check MT5 delay
    t = datetime.now()
    minutes_900 = 60*(t.hour-9) + t.minute
    mt5_hour, mt5_minute = (df2["minute"].iloc[-1] // 100), (df2["minute"].iloc[-1] % 100)
    mt5_minutes_900 = 60*(mt5_hour-9) + mt5_minute
    if minutes_900 > mt5_minutes_900 + 1:
        raise Exception(f"ERROR: platform delay with last minute {df2['minute'].iloc[-1]}")
    
    if indicators:
        df3 = update_indicators(df2.copy())
    else:
        df3 = df2.copy()
    
    return df2, df3

def update_df(df2, last_candle):
    df_candle, _ = get_df(last_candle, indicators=False)
    if len(df2) + len(df_candle) > 0:
        df2 = df2.iloc[-maximum_length+1:].append(df_candle, ignore_index=True)
        df3 = update_indicators(df2.copy())
        log(f"Updated df to {len(df2)} rows with last time {df2['minute'].iloc[-1]}")
        return df2, df3
    else:
        if len(df2) == 0:
            log(f"WARNING: current df has 0 rows)")
        elif len(df_candle) == 0:
            log(f"WARNING: failed to update df ({len(df2)} rows) with new candle")
        return df2, df2.copy()

# Functions for email handling

In [ ]:
def check_email(text, verbose=True):
    try:
        imap_server = imaplib.IMAP4_SSL(host="imap.gmail.com", port=993)
        imap_server.login(email_address, email_password)
        imap_server.select()
        if imap_server.state == "SELECTED":
            respose_code, message_numbers_raw = imap_server.search(None, 'BODY', text)
            
            imap_server.logout()
            if respose_code == "OK":                
                message_numbers = message_numbers_raw[0].split()
                if len(message_numbers):
                    log(f"Email about {text} found")
                    return True, True
                if verbose:
                    log(f"Email about {text} was NOT found")
                return True, False
        else:
            log(f"WARNING: check_email with imap_server.state = {imap_server.state}")
            try:
                imap_server.logout()
            except Exception as e:
                log(f"ERROR: check_email failed in imap_server close/logout with Exception: \n{e}")
            return False, False
            
    except Exception as e:
        log(f"ERROR: check_email failed with Exception: \n{e}")
        try:
            imap_server.logout()
        except Exception as ee:
            log(f"ERROR: check_email failed in imap_server close/logout with Exception: \n{ee}")
        return False, False

def send_email(subject, body):
    try:
        log(f"Sending email about {subject}")
        datetime_now = datetime.now()
        timestamp = datetime_now.strftime('%Y-%m-%d %H:%M:%S')
        subject = f"({timestamp}) {subject}"
        
        # Craft the email by hand
        from_email = email_address  # or simply the email address
        to_emails = [email_address]
        headers = f"From: {from_email}\r\n"
        headers += f"To: {', '.join(to_emails)}\r\n" 
        headers += f"Subject: {subject}\r\n"
        email_message = headers + "\r\n" + str(body)  # Blank line needed between headers and body

        # Connect, authenticate, and send mail
        smtp_server = SMTP_SSL("imap.gmail.com", port=SMTP_SSL_PORT)
        smtp_server.set_debuglevel(1)  # Show SMTP server interactions
        smtp_server.login(email_address, email_password)
        try:
            smtp_server.sendmail(from_email, to_emails, email_message)
        except Exception as ee:
            log(f"WARNING: original email text failed with Exception: \n{ee}")
            email_message = headers + "\r\n" + "Error text failed"
            smtp_server.sendmail(from_email, to_emails, email_message)
            

        # Disconnect
        smtp_server.quit()
        return True
    except Exception as e:
        log(f"ERROR: send_email failed with Exception: \n{e}")
        return False

In [ ]:
def send_daily_log():
    datetime_now = datetime.now()
    day_now = datetime_now.strftime('%Y-%m-%d')
    filename = f"{day_now} WIN.txt"
    path = os.path.join("logs", filename)
    f = open(path, "r")
    text = str(f.read()).encode('utf-8')
    success = send_email(f"DAILY LOG {day_now}", text)
    while not success:
        success = send_email(f"DAILY LOG {day_now}", text)
    f.close() 

# Functions for managing the positions

In [ ]:
def send_order(symbol, action, lot, comment, force=False):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py
    '''
    
    ask, bid = get_last_tick(symbol)
    if action == 'L':
        price, trade_type = ask, mt5.ORDER_TYPE_BUY
    elif action == 'S':
        price, trade_type = bid, mt5.ORDER_TYPE_SELL
    
    filling_type = mt5.ORDER_FILLING_IOC if not force else mt5.ORDER_FILLING_RETURN

    request = {
        "action": mt5.TRADE_ACTION_DEAL, # ordem a mercado
        "symbol": symbol, 
        "volume": float(lot),
        "type": trade_type,
        "price": price,
        "deviation": deviation,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
        "type_filling": filling_type, # order executed up to available volume
    }
    
    # Check the trading request before sending
    result_check = mt5.order_check(request)
    if result_check.retcode != 0:
        # Report warning and try to reconnect
        log(f"WARNING: order_check failed with retcode {result_check.retcode}: \n{result_check}")
        mt5.shutdown()
        mt5.initialize()
        result_check2 = mt5.order_check(request)
        log(f"WARNING: order_check failed AGAIN with retcode {result_check2.retcode}: \n{result_check2}")
    
    # Send actual order with updated price
    ask, bid = get_last_tick(symbol)
    request["price"] = ask if action == 'L' else bid
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        log(f"WARNING: order_send failed with retcode {result.retcode}: \n{result}")
        
        # Reconnect and send order again with updated price
        mt5.shutdown()
        mt5.initialize()
        ask, bid = get_last_tick(symbol)
        request["price"] = ask if action == 'L' else bid
        result2 = mt5.order_send(request)
        if result2.retcode != mt5.TRADE_RETCODE_DONE:
            log(f"ERROR: order_send failed AGAIN with retcode {result2.retcode}: \n{result2}")
            log(f"Failed trade request: \n{request}")
            raise Exception("ERROR: order_send failed twice")
    
    return result, request 

In [ ]:
def get_contracts(atr, ht):
    risk = atr * np.sqrt(ht)
    if (np.isnan(risk)) or (risk < 1e-5):
        risk = threshold - 1e-5
    num = min(int(threshold/risk), limit)
    return num

def rebalance(name, force=False):
    all_orders = mt5.orders_get()
    if (len(all_orders) != 0):
        raise Exception(f"There are {len(all_orders)} open orders: \n{all_orders}")
    
    # Update open positions
    epoch_now = time.time()
    for mode in ["L", "S"]:
        trades[mode] = trades[mode][trades[mode] > epoch_now]
    for key in list(strat_dict.keys()):
        if strat_dict[key] <= epoch_now:
            strat_dict[key] = 0
    
    # Get desired net position and cap exposure
    desired_net = len(trades["L"]) - len(trades["S"])
    desired_net = min(exposure, desired_net)
    desired_net = max(-exposure, desired_net)
    
    # Get current position
    positions = mt5.positions_get()
    my_positions = []
    for position in positions:
        if position.symbol == name:
            my_positions.append(position)
    
    num_positions = len(my_positions)
    if num_positions not in [0, 1]:
        raise Exception(f"There are {num_positions} open positions: \n{my_positions}")
    
    if num_positions == 0:
        current_net = 0
    else:
        mult = 1 if my_positions[0].type == 0 else -1
        current_net = mult * int(my_positions[0].volume)
    
    # Get desired change and associated action
    change = desired_net - current_net
    action = None
    if change > 0:
        action = "L"
    elif change < 0:
        action = "S"
    else:
        log(f"Current net position: {current_net} | KEEP")
    
    # Send order if there is one
    if action is not None:
        comment = f"{action}: {current_net} --> {desired_net}"
        log(f"Current net position: {current_net} | {comment}")
        result, request = send_order(name, action, abs(change), comment, force=force)
        log("Trade was placed successfully")
    return

In [ ]:
def run_strategy(name, df3):
    if len(df3) >= 4:
        
        columns = df3.columns.to_list()
        # Check new entries
        for mode in ["L", "S"]:
            df_mode = df_strat[df_strat.MODE == mode]
            for i, strat, ht in zip(df_mode.index, df_mode["STRAT"], df_mode["HT"]):
                
                # Conditions to be able to execute strategy
                minute = df3.iloc[-1]["minute"]
                remaining_time = 60*(17 - minute//100) + (30 - minute % 100)
                if (ht < remaining_time) and (strat_dict[i] == 0):
                    
                    mask = True
                    entries = strat.split("||")
                    for entry in entries:
                        criteria = entry.replace("|", "").strip()
                        
                        for word in criteria.split(" "):
                            indicator = word.strip()
                            if indicator in columns:
                                exec(f"{indicator} = df3.iloc[-1][indicator]")
                        
                        mask = mask & ne.evaluate(criteria)
                    
                    if mask:
                        # correct for risk
                        atr = df3[f"ATR{ht}"].iloc[-1]
                        num = get_contracts(atr, ht)
                        log(f"Opening {num}x {mode}{ht} | {strat}")
                        
                        end_time = 60 * (time.time()//60 + ht)
                        trades[mode] = np.append([end_time]*num, trades[mode])
                        strat_dict[i] = end_time
        rebalance(name)
    return

# Functions for execution control

In [ ]:
def finish_day(name, error=False):
    log("Liquidating all orders to close day")
    
    if error:
        success, error_sent = check_email("MT5_SHUTDOWN")
        if not error_sent:
            send_email("MT5_SHUTDOWN", "")
    
    # Get open orders and positions
    positions, orders = mt5.positions_get(), mt5.orders_get()
    if (positions is not None) and (orders is not None):
        total = len(positions) + len(orders)
    else:
        total = 1
    
    started = time.time()
    while total > 0:
        # liquidate all orders
        restart(reset_trades=True)
        rebalance(name, force=True)
        time.sleep(15)
        positions, orders = mt5.positions_get(), mt5.orders_get()
        if (positions is not None) and (orders is not None):
            total = len(positions) + len(orders)
        else:
            total = 1
        
        if time.time() - started > 60:
            success, email_sent = check_email("MT5_CLOSING_FAILED")
            if not email_sent:
                email_sent = send_email("MT5_CLOSING_FAILED", f"orders = {orders} \n\npositions = {positions}")
    
    log("Shutting down for today!")
    mt5.shutdown()
    send_daily_log()
    return

In [ ]:
def check_start():
    while True:
        success, shutdown = check_email("MT5_SHUTDOWN")
        t = datetime.now()
        hour, minute, second = t.hour, t.minute, t.second
        composite = hour*100 + minute
        if (composite < 903) or (composite > 1729) or (t.weekday() in [5, 6]) or shutdown:
            log("Checked start")
            time.sleep(60 * (63 - minute) + 61 - second)
        else:
            log(f"Started code for the day!")
            connect()
            return True

def check_end(name):
    success, shutdown = check_email("MT5_SHUTDOWN", verbose=False)
    if not success:
        log("WARNING: check_email failed with 'MT5_SHUTDOWN'")
        
    t = datetime.now()
    hour, minute = t.hour, t.minute
    composite = hour*100 + minute
    
    if (composite < 903) or (composite >= 1730) or shutdown:
        finish_day(name)
        return False
    else:
        return True

In [ ]:
def restart(reset_trades=False):
    log("WARNING: reconnecting to MT5 due to errors")
    
    # Reconnect to MT5
    mt5.shutdown()
    time.sleep(5)
    connect()
    
    if reset_trades:
        log("WARNING: resetting trades due to errors")
        # Reset trades dict
        trades["L"] = np.array([])
        trades["S"] = np.array([])

        # Reset strat_dict
        for i in df_strat.index.to_numpy():
            strat_dict[i] = 0
        
def manage_crisis(name, last_fail, e):
    log(f"ERROR: caught Exception while running: \n{e}")
    
    if (time.time() - last_fail)//60 > 3*tolerance_minutes:
        finish_day(name, error=True)
        return False
    
    elif (time.time() - last_fail)//60 > tolerance_minutes:
        success, email_sent = check_email("MT5_CODE_FAILING")
        while not email_sent:
            email_sent = send_email("MT5_CODE_FAILING", e)
            time.sleep(60)
            if (time.time() - last_fail) > 2*tolerance_minutes:
                finish_day(name, error=True)
                return False
    
    restart()
    return True

In [ ]:
def minute_loop(df2, current_epoch, name):
    enter_time = time.time()
    while 1:
        last_candle = mt5.copy_rates_from(name, mt5.TIMEFRAME_M1, utc_from, 2)
        if last_candle is not None:
            last_candle = last_candle[:-1]
            last_epoch = int(last_candle[0][0]) #+ 1000 #############

            if last_epoch > current_epoch + 1:
                if last_epoch - current_epoch != 60:
                    log(f"WARNING: epoch jumped {last_epoch - current_epoch}s from {current_epoch} to {last_epoch}")
                df2, df3 = update_df(df2, last_candle)
                run_strategy(name, df3)
                current_epoch = last_epoch
                return df2, df3, current_epoch

            elif time.time() - enter_time > 50:
                raise Exception(f"ERROR: 50s elapsed without getting new data")
        
        elif time.time() - enter_time > 50:
            raise Exception(f"ERROR: 50s elapsed getting only NONE data")
        
def second_loop(name):
    while 1:
        # check for SL and TP
        time.sleep(0.1)
        t = datetime.now()
        if t.second + t.microsecond/1000000 > 59.8:
            return
    
    #time.sleep(59.9 - t.second - t.microsecond / 1000000.0)
    #return

In [ ]:
def main():
    last_fail = 1e12
    RUNNING = False
    while True:
        if not RUNNING:
            RUNNING = check_start()
        else:
            try:
                name = get_contract("WIN*")
                if name is not None:
                    df, df_ind, current_epoch = initialize_df(name)
                    t = datetime.now()
                    time.sleep(60 - t.second)
                    while RUNNING:
                        df, df_ind, current_epoch = minute_loop(df, current_epoch, name)
                        RUNNING = check_end(name)
                        t = datetime.now()
                        time.sleep(max(0, 59 - t.second - t.microsecond / 1000000))
                        last_fail = 1e12
            
            except Exception as e:
                if last_fail > time.time():
                    last_fail = time.time()
                RUNNING = manage_crisis(name, last_fail, e)

# Output of a real execution

### Internet connectivity was a serious issue during this run, so one can see the error handling in action in the log printed below. Ibovespa futures open at 9am, so the bot "wakes up" at 9:04 (you can see this happening below for 2020-09-02) and starts trading from 9:05 until 5:30pm (17:30), when it liquidates all positions, emails the owner a report of the day and then "sleeps" until the next trading day (this can be seen below for 2020-09-01). This is all done automatically as the detailed log below shows. 

In [ ]:
main()


(2020-09-01 14:20:49) Email about MT5_SHUTDOWN was NOT found

(2020-09-01 14:20:49) Started code for the day!

(2020-09-01 14:20:49) New connection established with MT5: 
 TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2572, maxbars=100000, codepage=1252, ping_last=11059, community_balance=0.0, retransmission=0.12015750767270833, company='Modal DTVM Ltda.', name='ModalMais MetaTrader 5', language='Portuguese (Brazil)', path='C:\\Program Files\\ModalMais MetaTrader 5', ...)

(2020-09-01 14:20:49) Next contract is WINV20 with 43 remaining days


D:\Anaconda\lib\site-packages\ta\trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
D:\Anaconda\lib\site-packages\ta\trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])



(2020-09-01 14:20:50) Initialized df with 205 rows and last time 2020-09-01 14:19:00

(2020-09-01 14:21:01) Updated df to 205 rows with last time 1420

(2020-09-01 14:21:02) Current net position: 0 | KEEP

(2020-09-01 14:22:01) Updated df to 205 rows with last time 1421

(2020-09-01 14:22:01) Current net position: 0 | KEEP

(2020-09-01 14:23:01) Updated df to 205 rows with last time 1422

(2020-09-01 14:23:01) Current net position: 0 | KEEP

(2020-09-01 14:24:01) Updated df to 205 rows with last time 1423

(2020-09-01 14:24:02) Current net position: 0 | KEEP

(2020-09-01 14:25:01) Updated df to 205 rows with last time 1424

(2020-09-01 14:25:01) Current net position: 0 | KEEP

(2020-09-01 14:26:02) Updated df to 205 rows with last time 1425

(2020-09-01 14:26:03) Current net position: 0 | KEEP

(2020-09-01 14:27:01) Updated df to 205 rows with last time 1426

(2020-09-01 14:27:01) Current net position: 0 | KEEP

(2020-09-01 14:28:02) Updated df to 205 rows with last time 1427

(2020-0


(2020-09-01 15:13:01) Updated df to 205 rows with last time 1512

(2020-09-01 15:13:01) Current net position: -2 | KEEP

(2020-09-01 15:14:01) Updated df to 205 rows with last time 1513

(2020-09-01 15:14:01) Current net position: -2 | KEEP

(2020-09-01 15:15:01) Updated df to 205 rows with last time 1514

(2020-09-01 15:15:01) Current net position: -2 | KEEP

(2020-09-01 15:16:01) Updated df to 205 rows with last time 1515

(2020-09-01 15:16:01) Current net position: -2 | L: -2 --> -1

(2020-09-01 15:16:02) Trade was placed successfully

(2020-09-01 15:17:01) Updated df to 205 rows with last time 1516

(2020-09-01 15:17:01) Opening 1x S40 | | X_CMF14_035 == 1 |

(2020-09-01 15:17:01) Current net position: -1 | S: -1 --> -2

(2020-09-01 15:17:01) Trade was placed successfully

(2020-09-01 15:18:01) Updated df to 205 rows with last time 1517

(2020-09-01 15:18:01) Opening 1x S30 | | CMF18 > 0.35 || CMF18 <= 0.4 |

(2020-09-01 15:18:01) Opening 1x S17 | | RSI_24 > 70 || RSI_24 <= 80 |




(2020-09-01 16:03:01) Updated df to 205 rows with last time 1602

(2020-09-01 16:03:01) Current net position: -1 | KEEP

(2020-09-01 16:04:01) Updated df to 205 rows with last time 1603

(2020-09-01 16:04:01) Opening 1x L23 | | BIN_AROON60 == -1 |

(2020-09-01 16:04:01) Current net position: -1 | L: -1 --> 0

(2020-09-01 16:04:02) Trade was placed successfully

(2020-09-01 16:05:01) Updated df to 205 rows with last time 1604

(2020-09-01 16:05:01) Current net position: 0 | KEEP

(2020-09-01 16:05:22) ERROR: check_email failed with Exception: 
[WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto

(2020-09-01 16:05:22) ERROR: check_email failed in imap_server close/logout with Exception: 
local variable 'imap_server' referenced before assignment

(2020-09-01 16:05:22) WARNING: check_email failed with 'MT5_SHUTDOWN'

(2020-09-01 16:06:01) Updated df to 205 rows with last time 1605

(2020-09-01 16:06:01) Current net position: 0 | KEEP

(2020-09-01 16:06:02


(2020-09-01 16:55:01) Updated df to 205 rows with last time 1654

(2020-09-01 16:55:01) Current net position: 1 | KEEP

(2020-09-01 16:56:01) Updated df to 205 rows with last time 1655

(2020-09-01 16:56:01) Current net position: 1 | KEEP

(2020-09-01 16:57:01) Updated df to 205 rows with last time 1656

(2020-09-01 16:57:01) Current net position: 1 | KEEP

(2020-09-01 16:58:01) Updated df to 205 rows with last time 1657

(2020-09-01 16:58:01) Current net position: 1 | S: 1 --> 0

(2020-09-01 16:58:01) Trade was placed successfully

(2020-09-01 16:59:01) Updated df to 205 rows with last time 1658

(2020-09-01 16:59:01) Current net position: 0 | S: 0 --> -1

(2020-09-01 16:59:01) Trade was placed successfully

(2020-09-01 17:00:01) Updated df to 205 rows with last time 1659

(2020-09-01 17:00:01) Current net position: -1 | KEEP

(2020-09-01 17:01:01) Updated df to 205 rows with last time 1700

(2020-09-01 17:01:01) Current net position: -1 | KEEP

(2020-09-01 17:02:01) Updated df to 20

send: 'ehlo [192.168.0.24]\r\n'
reply: b'250-smtp.gmail.com at your service, [2804:14c:161:8123:ed94:83a6:d339:e722]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [2804:14c:161:8123:ed94:83a6:d339:e722]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nSMTPUTF8'
send: 'AUTH PLAIN AGhlbnJpcXVlbm1vbnRlaXJvQGdtYWlsLmNvbQBIOTlsMDRvMTJA\r\n'
reply: b'235 2.7.0 Accepted\r\n'
reply: retcode (235); Msg: b'2.7.0 Accepted'
send: 'mail FROM:<henriquenmonteiro@gmail.com> size=103550\r\n'
reply: b'250 2.1.0 OK k48sm3153218qtk.44 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.0 OK k48sm3153218qtk.44 - gsmtp'
send: 'rcpt TO:<henriquenmonteiro@gmail.com>\r\n'
reply: b'250 2.1.5 OK k4

reply: b'250 2.0.0 OK  1598992207 k48sm3153218qtk.44 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1598992207 k48sm3153218qtk.44 - gsmtp'
data: (250, b'2.0.0 OK  1598992207 k48sm3153218qtk.44 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection k48sm3153218qtk.44 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection k48sm3153218qtk.44 - gsmtp'



(2020-09-01 17:31:01) Email about MT5_SHUTDOWN found

(2020-09-01 17:31:01) Checked start

(2020-09-01 19:28:43) Email about MT5_SHUTDOWN was NOT found

(2020-09-01 19:28:43) Checked start

(2020-09-02 01:57:59) Email about MT5_SHUTDOWN was NOT found

(2020-09-02 01:57:59) Checked start

(2020-09-02 08:53:09) Email about MT5_SHUTDOWN was NOT found

(2020-09-02 08:53:09) Checked start

(2020-09-02 09:04:02) ERROR: check_email failed with Exception: 
[WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto

(2020-09-02 09:04:02) ERROR: check_email failed in imap_server close/logout with Exception: 
local variable 'imap_server' referenced before assignment

(2020-09-02 09:04:02) Started code for the day!

(2020-09-02 09:04:02) New connection established with MT5: 
 TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notificati

KeyboardInterrupt: 